In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
sys.path.append('../experiments')
import seaborn as sns
import os
import pandas as pd
from copy import deepcopy
from matplotlib import pyplot as plt
from os.path import join
import numpy as np
import imodelsx.process_results
from neuro.features import qa_questions, feature_spaces
from neuro.data import story_names
from neuro.features.stim_utils import load_story_wordseqs, load_story_wordseqs_huge
import random
import json
import joblib
from neuro.features.qa_questions import get_questions, get_merged_questions_v3_boostexamples
from tqdm import tqdm
from collections import defaultdict

In [1]:
# Convert each question in this list to its core concept and list short phrases with that concept. Return answers as a dictionary with th question as the key and the concept/examples as a dictionary value.

questions = [
    'Does the sentence describe a personal reflection or thought?',
    'Does the sentence contain a proper noun?',
    'Does the sentence describe a physical action?',
    'Does the sentence describe a personal or social interaction that leads to a change or revelation?',
    'Does the sentence involve the mention of a specific object or item?',
    # 'Does the sentence involve a description of physical environment or setting?',
    'Does the sentence describe a relationship between people?',
    # 'Does the sentence mention a specific location?',
    # 'Is time mentioned in the input?',
    'Is the sentence abstract rather than concrete?',
    "Does the sentence express the narrator's opinion or judgment about an event or character?",
    'Is the input related to a specific industry or profession?',
    'Does the sentence include dialogue?',
    # 'Does the sentence describe a visual experience or scene?',
    'Does the input involve planning or organizing?',
    # 'Does the sentence involve spatial reasoning?',
    'Does the sentence involve an expression of personal values or beliefs?',
    'Does the sentence contain a negation?',
    'Does the sentence describe a sensory experience?',
    'Does the sentence include technical or specialized terminology?',
    'Does the input contain a number?',
    'Does the sentence contain a cultural reference?',
    # 'Does the text describe a mode of communication?',
    # 'Does the input include a comparison or metaphor?',
    # 'Does the sentence express a sense of belonging or connection to a place or community?',
    # 'Does the sentence describe a specific sensation or feeling?',
    # 'Does the text include a planning or decision-making process?',
    # 'Does the sentence include a personal anecdote or story?',
    # 'Does the sentence involve a discussion about personal or social values?',
    # 'Does the text describe a journey?',
    # 'Does the input contain a measurement?',
    # 'Does the sentence describe a physical sensation?',
    # 'Does the sentence include a direct speech quotation?',
    # 'Is the sentence reflective, involving self-analysis or introspection?',
    # 'Does the input describe a specific texture or sensation?'
]

# Example answer for the first two questions

concept_and_examples_dict = {
    'Does the sentence describe a personal reflection or thought?': {
        'concept': 'personal reflections or thoughts',
        'examples': [
            'I wonder if things would have been different',
            'I did the right thing',
            'I wonder what will happen next',
            'I remember when I was a child',
            'I believe that the world is a good place',
            'I feel like I am a good person',
            'she knew she was wrong',
            'I dream of a better world',
            'She really was silly',
        ]
    },
    'Does the sentence contain a proper noun?': {
        'concept': 'proper nouns',
        'examples': [
            'The Eiffel Tower',
            'The Beatles',
            'Google',
            'California',
            'Jupiter',
        ]
    },
}

In [2]:
concept_and_examples_dict = {
    'Does the sentence describe a personal reflection or thought?': {
        'concept': 'personal reflections or thoughts',
        'examples': [
            'I wonder if things would have been different',
            'I did the right thing',
            'I wonder what will happen next',
            'I remember when I was a child',
            'I believe that the world is a good place',
            'I feel that I am a good person',
            'she knew she was wrong',
            'I dream of a better world',
            'She really was silly',
        ]
    },
    'Does the sentence contain a proper noun?': {
        'concept': 'proper nouns',
        'examples': [
            'The Eiffel Tower',
            'The Beatles',
            'Google',
            'California',
            'Jupiter',
        ]
    },
    'Does the sentence describe a physical action?': {
        'concept': 'physical actions',
        'examples': [
            'She ran across the street',
            'He picked up the book',
            'They jumped into the pool',
            'She writes a letter',
            'He is eating an apple',
            'She danced in the rain',
        ]
    },
    'Does the sentence describe a personal or social interaction that leads to a change or revelation?': {
        'concept': 'personal or interactions interactions',
        'examples': [
            'They argued and then he realized his mistake',
            'After their conversation, she decided to change her career',
            'He confessed his love and she felt a surge of joy',
            'The debate made him rethink his beliefs',
            'Their friendship transformed over the years',
        ]
    },
    'Does the sentence involve the mention of a specific object or item?': {
        'concept': 'specific objects or items',
        'examples': [
            'The red ball',
            'A shiny new car',
            'The ancient artifact',
            'Her favorite book',
            'The broken vase',
            'golden ring',
        ]
    },
    'Does the sentence involve a description of physical environment or setting?': {
        'concept': 'environment or setting descriptions',
        'examples': [
            'The room was dimly lit',
            'The forest was dense and dark',
            'The city skyline at sunset',
            'The beach was serene',
            'The mountains towered above them',
            'The sidewalk was cracked and uneven',
        ]
    },
    'Does the sentence describe a relationship between people?': {
        'concept': 'relationships between people',
        'examples': [
            'She was his best friend',
            'They were colleagues for ten years',
            'He is her brother',
            'They are rivals',
            'She is my mentor',
            'He is her confidant',
        ]
    },
    'Does the sentence mention a specific location?': {
        'concept': 'specific locations',
        'examples': [
            'Paris',
            'The Grand Canyon',
            'Mount Everest',
            'Central Park',
            'The Pacific Ocean',
            'The North Pole',
        ]
    },
    'Is time mentioned in the input?': {
        'concept': 'mentions of time',
        'examples': [
            'Yesterday',
            'In 2025',
            'At dawn',
            'Later that evening',
            'Three years ago',
            'At midnight',
        ]
    },
    'Is the sentence abstract rather than concrete?': {
        'concept': 'abstract descriptions',
        'examples': [
            'Freedom is priceless',
            'Love conquers all',
            'The concept of justice',
            'She felt an overwhelming sense of peace',
            'The idea of infinity',
            'He believed in the power of dreams',
        ]
    },
    "Does the sentence express the narrator's opinion or judgment about an event or character?": {
        'concept': 'opinions or judgments',
        'examples': [
            'He was the best player on the team',
            'That was a terrible idea',
            'She is the kindest person I know',
            'It was a spectacular event',
            'He acted foolishly',
            'She was a brilliant actress',
        ]
    },
    'Is the input related to a specific industry or profession?': {
        'concept': 'industry or profession',
        'examples': [
            'The tech industry',
            'Medical field',
            'Financial sector',
            'Educational system',
            'Construction business',
        ]
    },
    'Does the sentence include dialogue?': {
        'concept': 'dialogue',
        'examples': [
            '"Hello, how are you?" she asked',
            '"I think we should leave," he said',
            '"What are you doing here?"',
            '"I love this song," she exclaimed',
            '"Can you help me with this?"',
            '"Nice to see you"'
        ]
    },
    'Does the sentence describe a visual experience or scene?': {
        'concept': 'visual experiences or scenes',
        'examples': [
            'The sunset painted the sky in hues of orange',
            'The garden was full of blooming flowers',
            'The room was decorated with bright colors',
            'The lake reflected the mountains perfectly',
            'The fireworks lit up the night sky',
        ]
    },
    'Does the input involve planning or organizing?': {
        'concept': 'planning or organizing',
        'examples': [
            'They planned the event meticulously',
            'She organized her schedule for the week',
            'They arranged the chairs in a circle',
            'He mapped out the trip in detail',
            'They coordinated their efforts to complete the project',
        ]
    },
    'Does the sentence involve spatial reasoning?': {
        'concept': 'spatial reasoning',
        'examples': [
            'The table is to the left of the chair',
            'The ball rolled under the couch',
            'The cat jumped over the fence',
            'She placed the vase between the books',
            'The park is located north of the river',
        ]
    },
    'Does the sentence involve an expression of personal values or beliefs?': {
        'concept': 'personal values or beliefs',
        'examples': [
            'She believed in honesty above all',
            'He valued loyalty in his friends',
            'They upheld the tradition of their ancestors',
            'She thought kindness was the greatest virtue',
            'He held strong religious beliefs',
        ]
    },
    'Does the sentence contain a negation?': {
        'concept': 'negations',
        'examples': [
            'not good',
            'She did not like the movie',
            'He cannot attend the meeting',
            'They never spoke again',
            'I don’t think that’s true',
            'She isn’t coming to the party',
        ]
    },
    'Does the sentence describe a sensory experience?': {
        'concept': 'sensory experiences',
        'examples': [
            'The fragrance of the flowers filled the air',
            'The music was loud and exhilarating',
            'The texture of the fabric was soft',
            'The taste of the chocolate was rich and creamy',
            'The sight of the ocean was breathtaking',
        ]
    },
    'Does the sentence include technical or specialized terminology?': {
        'concept': 'technical or specialized terminology',
        'examples': [
            'The CPU processes data quickly',
            'The patient exhibited symptoms of anemia',
            'The stock market fluctuates daily',
            'The curriculum includes advanced calculus',
            'The software uses a complex algorithm',
        ]
    },
    'Does the input contain a number?': {
        'concept': 'numbers',
        'examples': [
            'Three cats',
            '50% discount',
            'Seven miles',
            '152',
            'Fourteen days',
        ]
    },
    'Does the sentence contain a cultural reference?': {
        'concept': 'cultural references',
        'examples': [
            'Like Romeo and Juliet',
            'An Achilles’ heel',
            'He was a real Scrooge',
            'The Mona Lisa',
            'A Herculean task',
            'Taylor Swift music'
        ]
    },
    'Does the text describe a mode of communication?': {
        'concept': 'modes of communication',
        'examples': [
            'They communicated via email',
            'She wrote him a letter',
            'They talked on the phone',
            'He sent a text message',
            'She used sign language',
        ]
    },
    'Does the input include a comparison or metaphor?': {
        'concept': 'comparisons or metaphors',
        'examples': [
            'She was as brave as a lion',
            'The night was as dark as coal',
            'Life is a journey',
            'He is a shining star',
            'Time is money',
        ]
    },
    'Does the sentence express a sense of belonging or connection to a place or community?': {
        'concept': 'sense of belonging or connection',
        'examples': [
            'She felt at home in the village',
            'He belonged to the local club',
            'They were part of a close-knit community',
            'She identified with the city’s culture',
            'He felt a connection to the land',
        ]
    },
    'Does the sentence describe a specific sensation or feeling?': {
        'concept': 'specific sensations or feelings',
        'examples': [
            'The sting of the bee was sharp',
            'She felt a wave of relief',
            'The warmth of the sun on her skin',
            'He felt a chill down his spine',
            'The pain in his leg was intense',
        ]
    },
    'Does the text include a planning or decision-making process?': {
        'concept': 'planning or decision-making process',
        'examples': [
            'They decided to move forward with the project',
            'She planned her career path carefully',
            'They weighed the pros and cons',
            'He chose the best option available',
            'They strategized their next move',
        ]
    },
    'Does the sentence include a personal anecdote or story?': {
        'concept': 'personal anecdotes or stories',
        'examples': [
            'When I was a child, we used to visit the beach every summer',
            'He told a story about his first job',
            'She shared a memory from her wedding day',
            'They recounted their adventure in the mountains',
            'I remember the time we got lost in the city',
        ]
    },
    'Does the sentence involve a discussion about personal or social values?': {
        'concept': 'discussion about values',
        'examples': [
            'They debated the importance of freedom',
            'She talked about the need for equality',
            'He emphasized the value of hard work',
            'They discussed the meaning of justice',
            'She highlighted the significance of kindness',
        ]
    },
    'Does the text describe a journey?': {
        'concept': 'journeys',
        'examples': [
            'They traveled across Europe',
            'The expedition took them to uncharted territories',
            'He embarked on a spiritual journey',
            'She journeyed through her past memories',
            'The voyage was long and arduous',
        ]
    },
    'Does the input contain a measurement?': {
        'concept': 'measurements',
        'examples': [
            '10 meters long',
            'Weighs 5 kilograms',
            'Three cups of flour',
            'Two hours later',
            'Temperature of 20 degrees',
        ]
    },
    'Does the sentence describe a physical sensation?': {
        'concept': 'physical sensations',
        'examples': [
            'The prickling sensation on her skin',
            'He felt the cool breeze on his face',
            'The burning sensation in his throat',
            'She felt a throbbing pain in her head',
            'His muscles ached after the workout',
        ]
    },
    'Does the sentence include a direct speech quotation?': {
        'concept': 'direct speech quotations',
        'examples': [
            '"I can’t believe this is happening," she said',
            '"What do you think?" he asked',
            '"I’ll be there soon," she promised',
            '"This is amazing!" he exclaimed',
            '"Why did you do that?" she inquired',
        ]
    },
    'Is the sentence reflective, involving self-analysis or introspection?': {
        'concept': 'self-analysis or introspection',
        'examples': [
            'He wondered about his choices in life',
            'She reflected on her past mistakes',
            'He analyzed his feelings of guilt',
            'She questioned her purpose in life',
            'He pondered his next steps',
        ]
    },
    'Does the input describe a specific texture or sensation?': {
        'concept': 'textures or sensations',
        'examples': [
            'The smooth surface of the marble',
            'The rough bark of the tree',
            'The silky feel of the fabric',
            'The gritty sand under her feet',
            'The sticky residue on the table',
        ]
    }
}

In [3]:
rows = {
    'expl': [concept_and_examples_dict[q]['concept'] for q in questions],
    'top_ngrams_module_correct': [concept_and_examples_dict[q]['examples'] for q in questions],
    'question': questions,
    'subject': ['UTS02'] * len(questions),
    'module_num': ['qa'] * len(questions),
    'voxel_num': ['qa'] * len(questions),
}

In [21]:
pd.DataFrame(rows).to_pickle('rows_qa_may31.pkl')